# **Penting**
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Hapus simbol pagar (#) jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]
df = pd.read_csv("data_clustering_inverse.csv")


In [3]:
# Tampilkan 5 baris pertama dengan function head.
df.head()

,TransactionAmount,TransactionType,CustomerAge,AccountBalance,Channel,CustomerOccupation,Target
0,44.896593,Debit,70.0,50.598497,ATM,Doctor,2
1,52.042324,Debit,68.0,60.908340,ATM,Doctor,2
2,47.110457,Debit,19.0,45.841210,Online,Student,1
3,48.259023,Debit,26.0,54.720252,Online,Student,1
4,44.883965,Credit,45.0,53.361385,Online,Student,2


In [5]:
df2 = df.copy()

# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [6]:
# Pisahkan fitur dan target
X = df2.drop(columns=['Target'])  # Semua kolom kecuali Target
y = df2['Target']  # Kolom Target (label cluster)

In [7]:
# Encoding fitur kategorikal
# Asumsi fitur kategorikal: TransactionType, Channel, CustomerOccupation
categorical_cols = ['TransactionType', 'Channel', 'CustomerOccupation']
le_dict = {}  # Dictionary untuk menyimpan LabelEncoder per kolom

for col in categorical_cols:
    if col in X.columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        le_dict[col] = le  # Simpan encoder untuk penggunaan di masa depan
    else:
        print(f"Kolom {col} tidak ditemukan di X")

# Simpan LabelEncoder untuk kategorikal
for col, le in le_dict.items():
    joblib.dump(le, f'label_encoder_{col}.pkl')
    print(f"LabelEncoder untuk {col} disimpan ke 'label_encoder_{col}.pkl'")

# Fitur numerik (tidak perlu scaling untuk Random Forest)
numeric_cols = ['TransactionAmount', 'AccountBalance', 'CustomerAge']
for col in numeric_cols:
    if col not in X.columns:
        print(f"Kolom {col} tidak ditemukan di X")

LabelEncoder untuk TransactionType disimpan ke 'label_encoder_TransactionType.pkl'
LabelEncoder untuk Channel disimpan ke 'label_encoder_Channel.pkl'
LabelEncoder untuk CustomerOccupation disimpan ke 'label_encoder_CustomerOccupation.pkl'


In [8]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
print(f"Ukuran data pelatihan: {X_train.shape}")
print(f"Ukuran data pengujian: {X_test.shape}")


Ukuran data pelatihan: (2011, 6)
Ukuran data pengujian: (503, 6)


# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [9]:
# Buatlah model klasifikasi menggunakan Decision Tree
dt_model = DecisionTreeClassifier(random_state=0)
dt_model.fit(X_train, y_train)
print("Pelatihan model Decision Tree selesai")

Pelatihan model Decision Tree selesai


In [10]:
# Menyimpan Model
# import joblib
# joblib.dump(model, 'decision_tree_model.h5')
joblib.dump(dt_model, 'decision_tree_model.h5')
print("Model Decision Tree disimpan ke 'decision_tree_model.h5'")

Model Decision Tree disimpan ke 'decision_tree_model.h5'


# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [11]:
# Melatih model menggunakan algoritma klasifikasi selain Decision Tree.
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)
print("Pelatihan model Random Forest selesai")

Pelatihan model Random Forest selesai


In [22]:
# --- Evaluasi Decision Tree ---
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import seaborn as sns

print("\n--- Evaluasi Decision Tree ---")
# Prediksi
y_pred = dt_model.predict(X_test)

# Evaluasi metrik
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average=None)  # Per kelas
recall = recall_score(y_test, y_pred, average=None)  # Per kelas
f1 = f1_score(y_test, y_pred, average=None)  # Per kelas

# Simpan hasil per kelas
results = []
for i, class_label in enumerate(np.unique(y)):
    results.append({
        'Model': ['Decision Tree'],
        'Class': [class_label],
        'Accuracy': [accuracy],
        'Precision': [precision[i]],
        'Recall': [recall[i]],
        'F1-Score': [f1[i]]
    })

# Cetak classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))





--- Evaluasi Decision Tree ---
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       100
           1       1.00      1.00      1.00       132
           2       0.99      0.96      0.98        84
           3       0.99      1.00      1.00       187

    accuracy                           0.99       503
   macro avg       0.99      0.99      0.99       503
weighted avg       0.99      0.99      0.99       503



In [23]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.
# Evaluasi metrik
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# --- Evaluasi Random Forest ---
print("\n--- Evaluasi Random Forest ---")
# Prediksi)
y_pred = rf_model.predict(X_test)

# Evaluasi metrik
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average=None)  # Per kelas
recall = recall_score(y_test, y_pred, average=None)  # Per kelas
f1 = f1_score(y_test, y_pred, average=None)  # Per kelas

# Simpan hasil per kelas
results = []
for i, class_label in enumerate(np.unique(y)):
    results.append({
        'Model': ['Random Forest'],
        'Class': [class_label],
        'Accuracy': [accuracy],
        'Precision': [precision[i]],
        'Recall': [recall[i]],
        'F1-Score': [f1[i]]
    })

# Cetak classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))




--- Evaluasi Random Forest ---
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100
           1       1.00      1.00      1.00       132
           2       1.00      0.95      0.98        84
           3       0.98      1.00      0.99       187

    accuracy                           0.99       503
   macro avg       0.99      0.99      0.99       503
weighted avg       0.99      0.99      0.99       503



In [24]:
# Menyimpan Model Selain Decision Tree
# Model ini bisa lebih dari satu
# import joblib
# joblib.dump(___, 'explore_<Nama Algoritma>_classification.h5')
joblib.dump(rf_model, 'explore_random_forest_classification.h5')
print("Model Decision Tree disimpan ke 'explore_random_forest_classification.h5'")

Model Decision Tree disimpan ke 'explore_random_forest_classification.h5'


Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [ ]:
# Lakukan Hyperparameter Tuning dan Latih ulang.

In [ ]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.

In [ ]:
# Menyimpan Model hasil tuning
# import joblib
# joblib.dump(model_dt, 'tuning_classification.h5')